In [1]:
import pandas as pd
import numpy as np

In [7]:
import os
import openai
from dotenv import load_dotenv

In [10]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY") 
# print(openai.api_key)

In [11]:
# embedding model parameters
embedding_model = "text-embedding-ada-002" # second gen best model at the moment
embedding_encoding = "cl100k_base" # latest tokenizer for second gen models
max_tokens = 8000 # max tokens for second gen models and tokenizer above is 8191

In [2]:
# Load the embedding data
df = pd.read_csv("data/mplus_full_embeddings.csv")

In [3]:
# only if reading from csv to ensure correct type
df["embedding"] = df.embedding.apply(eval).apply(np.array)

In [12]:
#multi-lingual search 
from openai.embeddings_utils import get_embedding, cosine_similarity

# search through the symptoms for the most similar topic
def search_symptoms(df, symptoms, n=5, pprint=True):
    symptoms_embedding = get_embedding(symptoms, engine=embedding_model)
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, symptoms_embedding))
    results = (
        df.sort_values("similarity", ascending=False)
        .head(n)
        .combined.str.replace("Topic: ", "")
        .str.replace("Symptoms: ", "")
    )
    if pprint:
        for r in results:
            print(r[:200])
            print()
    return results

In [14]:
results = search_symptoms(df, "spanish back pain", n=5)

Title: Dolor de espalda Meta Desc: La lumbalgia es el dolor en la parte baja de la espalda.  El dolor de espalda es uno de los problemas médicos más comunes. Tratamientos, prevención aquí. Also Called

Title: Back Pain Meta Desc: Lumbago is pain on the lower part of your back. Back pain is very common. Treatment varies from medicines to surgery depending on the cause and kind of pain. Also Called: L

Title: Dolor Meta Desc: El dolor es una sensación desencadenada por el sistema nervioso. Infórmese sobre los diferentes tipos de dolor y cómo manejarlos aquí. Also Called:  Full Summary: <p>El dolor e

Title: Lesiones de la espalda Meta Desc: Las lesiones de la espalda más comunes son distenciones y torceduras, discos herniados y fractura de vértebras. Entérese sobre tratamientos y más aquí. Also Ca

Title: Spinal Stenosis Meta Desc: Spinal stenosis causes narrowing in your spine which puts pressure on your nerves and spinal chord. It can cause pain, numbness, and foot problems. Also Calle